<a href="https://colab.research.google.com/github/RealMyeong/AIFFELTHON/blob/main/JinMyeong/%EC%9D%BC%EB%B0%98%EB%8C%80%ED%99%94_%EC%B6%94%EA%B0%80_%ED%8C%8C%EC%9D%B4%ED%94%84%EB%9D%BC%EC%9D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 진행 순서

1. 데이터 불러오기
  - 일반 대화 데이터
  - 공격 대화 데이터
2. 키워드 추출
  - 공격 대화 데이터 키워드 추출
  - 일반 대화 데이터 키워드 추출
    - 화자 구분 x
    - 상담사 제외 고객만 키워드 추출
3. 키워드 비율에 따라 일반대화 추가

### 1. 데이터 불러오기

In [15]:
import pandas as pd
import numpy as np
import re

In [16]:
# 일반 대화 데이터
normal_conv = pd.read_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/norm_data.csv')

# 공격 대화 데이터
attack_conv = pd.read_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/add_biclass_train.csv')

In [17]:
attack_conv.columns

Index(['Unnamed: 0', 'class', 'conversation', 'labelling', 'label_sentence',
       'cleaned_sentence', 'spell_checked_conversation', 'binary_class'],
      dtype='object')

In [18]:
attack_conv.drop(['Unnamed: 0', 'labelling', 'conversation', 'cleaned_sentence'], axis=1, inplace=True)
attack_conv

,class,label_sentence,spell_checked_conversation,binary_class
0,직장 내 괴롭힘 대화,1:길동씨 이번에 이것좀 처리해요\n2:이거 제가 한게 아닌데요\n1:팀에서 내가 ...,길동 씨 이번에 이것 좀 처리해요\n이거 제가 한 게 아닌데요\n팀에서 내가 네가 ...,공격 대화
1,기타 괴롭힘 대화,1:야 너 앞니 왜 그렇게 튀어나왔냐?\n2:태어날때부터 그랬어 물어보지마.\n1:...,야 너 앞니 왜 그렇게 튀어나왔냐?\n태어날 때부터 그랬어 물어보지 마.\n아 그럼...,공격 대화
2,갈취 대화,1:원후야 학원 교재 사야되는데 8만원만 주면 안돼?\n2:미안 나 오늘은 진짜 돈...,원후야 학원 교재 사야 되는데 8만 원만 주면 안 돼?\n미안 나 오늘은 진짜 돈이...,공격 대화
3,갈취 대화,1:너 저번에 술먹은 날 기억해?\n2:아니 왜?\n1:야 300만원만 가져와.\n...,너 저번에 술 먹은 날 기억해?\n아니 왜?\n야 300만 원만 가져와.\n뭔 소리...,공격 대화
4,협박 대화,1:너 또 내말 무시하냐?\n1:이새끼 널 좆으로 보나본데?\n2:아냐 진짜 시간이...,너 또 내말 무시하냐?\n이 새끼 널 좆으로 보나 본데?\n아냐 진짜 시간이 없었어...,공격 대화
...,...,...,...,...
3908,기타 괴롭힘 대화,1:준하야 넌 대가리가 왜이렇게 크냐?\n2:내 머리가 뭐.\n1:밥먹으면 대가리만...,준하야 넌 대가리가 왜 이렇게 크냐?\n내 머리가 뭐.\n밥 먹으면 대가리만 크냐 ...,공격 대화
3909,갈취 대화,1:내가 지금 너 아들 김길준 데리고 있어. 살리고 싶으면 계좌에 1억만 보내\n2...,내가 지금 너 아들 김길준 데리고 있어. 살리고 싶으면 계좌에 1억만 보내\n예.?...,공격 대화
3910,직장 내 괴롭힘 대화,1:나는 씨 같은 사람 보면 참 신기하더라. 어떻게 저렇게 살지.\n1:왜 그래. ...,나는 씨 같은 사람 보면 참 신기하더라. 어떻게 저렇게 살지.\n왜 그래. 들리겠어...,공격 대화
3911,갈취 대화,1:누구맘대로 여기서 장사하래?\n2:이게 무슨일입니까?\n1:남의 구역에서 장사하...,누구 맘대로 여기서 장사하래?\n이게 무슨 일입니까?\n남의 구역에서 장사하려면 자...,공격 대화


In [19]:
attack_conv.rename(columns ={'spell_checked_conversation' : 'non_label_sentence'}, inplace=True)
attack_conv

,class,label_sentence,non_label_sentence,binary_class
0,직장 내 괴롭힘 대화,1:길동씨 이번에 이것좀 처리해요\n2:이거 제가 한게 아닌데요\n1:팀에서 내가 ...,길동 씨 이번에 이것 좀 처리해요\n이거 제가 한 게 아닌데요\n팀에서 내가 네가 ...,공격 대화
1,기타 괴롭힘 대화,1:야 너 앞니 왜 그렇게 튀어나왔냐?\n2:태어날때부터 그랬어 물어보지마.\n1:...,야 너 앞니 왜 그렇게 튀어나왔냐?\n태어날 때부터 그랬어 물어보지 마.\n아 그럼...,공격 대화
2,갈취 대화,1:원후야 학원 교재 사야되는데 8만원만 주면 안돼?\n2:미안 나 오늘은 진짜 돈...,원후야 학원 교재 사야 되는데 8만 원만 주면 안 돼?\n미안 나 오늘은 진짜 돈이...,공격 대화
3,갈취 대화,1:너 저번에 술먹은 날 기억해?\n2:아니 왜?\n1:야 300만원만 가져와.\n...,너 저번에 술 먹은 날 기억해?\n아니 왜?\n야 300만 원만 가져와.\n뭔 소리...,공격 대화
4,협박 대화,1:너 또 내말 무시하냐?\n1:이새끼 널 좆으로 보나본데?\n2:아냐 진짜 시간이...,너 또 내말 무시하냐?\n이 새끼 널 좆으로 보나 본데?\n아냐 진짜 시간이 없었어...,공격 대화
...,...,...,...,...
3908,기타 괴롭힘 대화,1:준하야 넌 대가리가 왜이렇게 크냐?\n2:내 머리가 뭐.\n1:밥먹으면 대가리만...,준하야 넌 대가리가 왜 이렇게 크냐?\n내 머리가 뭐.\n밥 먹으면 대가리만 크냐 ...,공격 대화
3909,갈취 대화,1:내가 지금 너 아들 김길준 데리고 있어. 살리고 싶으면 계좌에 1억만 보내\n2...,내가 지금 너 아들 김길준 데리고 있어. 살리고 싶으면 계좌에 1억만 보내\n예.?...,공격 대화
3910,직장 내 괴롭힘 대화,1:나는 씨 같은 사람 보면 참 신기하더라. 어떻게 저렇게 살지.\n1:왜 그래. ...,나는 씨 같은 사람 보면 참 신기하더라. 어떻게 저렇게 살지.\n왜 그래. 들리겠어...,공격 대화
3911,갈취 대화,1:누구맘대로 여기서 장사하래?\n2:이게 무슨일입니까?\n1:남의 구역에서 장사하...,누구 맘대로 여기서 장사하래?\n이게 무슨 일입니까?\n남의 구역에서 장사하려면 자...,공격 대화


In [20]:
normal_conv

,type,non_label_sentence,label_sentence,class,binary_class
0,고객센터,반갑습니다 입니다 무엇을 도와드릴까요\n여보세요 네 안녕하세요 거기 맞나요\n연결 ...,1:반갑습니다 입니다 무엇을 도와드릴까요\n2:여보세요 네 안녕하세요 거기 맞나요\...,일반 대화,일반 대화
1,고객센터,조직도 확인 후 연락드렸습니다\n정확한 문의는 부서에 문의를 해보시면 좋으실 듯 합...,1: 조직도 확인 후 연락드렸습니다\n1:정확한 문의는 부서에 문의를 해보시면 좋으...,일반 대화,일반 대화
2,고객센터,반갑습니다 입니다 무엇을 도와드릴까요\n제가 주택 임대차로 분쟁조정위원회쪽을 통화를...,1:반갑습니다 입니다 무엇을 도와드릴까요\n2:제가 주택 임대차로 분쟁조정위원회쪽을...,일반 대화,일반 대화
3,고객센터,반갑습니다 입니다 무엇을 도와 드릴까요\n여기요 인가요\n네 맞습니다 입니다\n네 ...,1:반갑습니다 입니다 무엇을 도와 드릴까요\n2:여기요 인가요\n1:네 맞습니다 입...,일반 대화,일반 대화
4,고객센터,네 입니다\n안녕하세요 입니다\n네 안녕하세요 무슨 일이시죠\n안녕하세요 주사님 한...,2:네 입니다\n1:안녕하세요 입니다\n2:네 안녕하세요 무슨 일이시죠\n1:안녕하...,일반 대화,일반 대화
...,...,...,...,...,...
47990,카카오,요즘에 엄마가 나 씻고 화장하면 남자 만나냐고 물어봄\n갑자기 집이랑 만 왔다 갔다...,1:요즘에 엄마가 나 씻고 화장하면 남자 만나냐고 물어봄\n2:갑자기 집이랑 만 왔...,일반 대화,일반 대화
47991,카카오,주말에 보드 타러 갈래 사상에 괜찮은 데 있음 잘 가르치던대\n뭐 바퀴 달린 거 그...,1:주말에 보드 타러 갈래 사상에 괜찮은 데 있음 잘 가르치던대\n2:뭐 바퀴 달린...,일반 대화,일반 대화
47992,카카오,우리 집 뒤에 중국집 가지마\n왜 너 거기 좋아했잖아 맛있다며\n다 다음 달까지 거...,1:우리 집 뒤에 중국집 가지마\n2:왜 너 거기 좋아했잖아 맛있다며\n1:다 다음...,일반 대화,일반 대화
47993,카카오,엄마 생신 선물 눈치 게임 실패했다 아\n왜 너희 오빠랑 같이 준비해서 드린다고 했...,1:엄마 생신 선물 눈치 게임 실패했다 아\n2:왜 너희 오빠랑 같이 준비해서 드린...,일반 대화,일반 대화


In [52]:
normal_conv.iloc[[1,5]]

,type,non_label_sentence,label_sentence,class,binary_class
1,고객센터,조직도 확인 후 연락드렸습니다\n정확한 문의는 부서에 문의를 해보시면 좋으실 듯 합...,1: 조직도 확인 후 연락드렸습니다\n1:정확한 문의는 부서에 문의를 해보시면 좋으...,일반 대화,일반 대화
5,고객센터,반갑습니다 입니다 무엇을 도와드릴까요\n제가 뭐 좀 여쭤보려고 하는데요\n제가 가지...,1:반갑습니다 입니다 무엇을 도와드릴까요\n2:제가 뭐 좀 여쭤보려고 하는데요\n2...,일반 대화,일반 대화


### 2. 키워드 추출

1. 공격 대화 키워드 추출


In [21]:
!pip install krwordrank

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [33]:
def keyword(class_name, data, n=20):
    genre = data[data['class']==class_name].reset_index(drop=True).label_sentence

    attack, damaged = [], []

    for e in genre:
        e = e.split('\n')
        for e_ in e:
            if e_[0] == '1':
                attack.append(e_[2:])
            else:
                damaged.append(e_[2:])

    from krwordrank.sentence import summarize_with_sentences

    texts = attack
    keywords_att, sents = summarize_with_sentences(texts, num_keywords=100, 
                                           num_keysents=10, max_length=10, diversity=0.5)

    texts = damaged
    keywords_dam, sents = summarize_with_sentences(texts, num_keywords=100, 
                                           num_keysents=10, max_length=10, diversity=0.5)

    print('{}의 가해자 :'.format(class_name),keywords_att.keys())
    print()
    print('{}의 피해자 :'.format(class_name),keywords_dam.keys())
    
    keywords_att = list(keywords_att.keys())
    keywords_dam = list(keywords_dam.keys())
    
    target_attack, target_damaged = [], []

    for i in range(100):
        if (keywords_att[i] not in keywords_dam) : target_attack.append(keywords_att[i])
        if len(target_attack) == n: break 

    for i in range(100):
        if (keywords_dam[i] not in keywords_att) : target_damaged.append(keywords_dam[i])
        if len(target_damaged) == n: break
    
    print()
    print('{}의 가해자(중복제거) :'.format(class_name),target_attack)
    print()
    print('{}의 피해자(중복제거) :'.format(class_name),target_damaged)

    return [target_attack, target_damaged]

In [37]:
extort_att, extort_vic = keyword('갈취 대화', attack_conv)

갈취 대화의 가해자 : dict_keys(['내가', '그럼', '아니', '지금', '그냥', '이거', '그래', '빌려', '내놔', '빨리', '진짜', '돈이', '우리', '친구', '오늘', '나도', '제가', '니가', '여기', '없어', '나오면', '있는', '당장', '너무', '그러', '어떻게', '있어', '좋은', '가져', '이번', '거기', '그거', '그렇게', '보내', '10', '뒤져서', '나한테', '얼마', '사람', '어이', '당신', '내일', '한번', '알아', '그건', '엄마', '이렇게', '얼른', '근데', '맞고', '이게', '가지고', '누가', '너가', '무슨', '말이', '많이', '싫으면', '돈좀', '바로', '돈을', '지갑', '전에', '주면', '하나', '줄래', '어디', '그런', '그게', '가방', '새끼', '생각', '줘봐', '너네', '저기', '말고', '잠깐', '하면', '조용히', '입금', '줄게', '다른', '다시', '뭐야', '내놓', '이건', '돌려', '이제', '필요', '없는', '학교', '좋게', '쓰고', '이리', '신고', '저희', '좋아', '하는', '거짓말', '이런'])

갈취 대화의 피해자 : dict_keys(['제발', '아니', '진짜', '없어', '이거', '안돼', '무슨', '네?', '내가', '돈이', '정말', '아.', '제가', '지금', '그건', '어떻게', '미안', '여기', '그래', '그게', '나도', '알겠', '네.', '그러', '그럼', '죄송', '엄마', '그거', '저희', '이건', '이게', '저번에도', '이번', '그런', '없는', '빌려', '빨리', '돌려', '우리', '근데', '그냥', '한번만', '너무', '이제', '이러', '오늘', '돈을', '안되', '얼마', '조금만', '있어', '갑자기', '혹시', '누구', '당장', '드릴'

In [39]:
work_att, work_vic = keyword('직장 내 괴롭힘 대화', attack_conv)

직장 내 괴롭힘 대화의 가해자 : dict_keys(['내가', '아니', '회사', '지금', '그럼', '오늘', '그래', '자네', '그렇게', '우리', '그냥', '이렇게', '김대리', '무슨', '사람', '하는', '이거', '일을', '진짜', '어떻게', '이번', '요즘', '하고', '너무', '제대로', '내일', '그런', '말이', '이런', '그러', '다른', '없어', '니가', '다시', '빨리', '아주', '보고', '누가', '휴가', '너가', '일이', '생각', '근데', '같이', '뭐가', '하면', '하나', '못하', '어디', '그게', '알아', '똑바로', '있어', '이게', '여기', '죄송', '하지', '해야', '가서', '업무', '상사', '어제', '앞으로', '있는', '이래서', '커피', '말고', '나한테', '얼마나', '그건', '없는', '들어', '일도', '뭐?', '아직', '너는', '그리고', '한번', '이제', '나는', '주말에', '승진', '나도', '부탁', '뭐야', '집에', '혼자', '맨날', '제가', '그거', '싶어?', '나랑', '같은', '어이', '시간', '자기', '혹시', '나가', '회식', '하라'])

직장 내 괴롭힘 대화의 피해자 : dict_keys(['제가', '네?', '죄송', '아니', '부장님', '무슨', '너무', '네.', '정말', '다시', '지금', '아.', '과장님', '아닙니다', '말씀', '저도', '그건', '그래도', '일이', '오늘', '그게', '팀장님', '이건', '저는', '그런', '하겠습니다.', '열심히', '사장님', '대리님', '알겠', '빨리', '업무', '진짜', '같습니다.', '그럼', '있어서', '아닌', '내일', '다른', '어떻게', '이번', '하고', '아직', '앞으로', '그렇게', '바로', '혹시', '그냥', '회사', '그러', '갑자기'

In [40]:
threat_att, threat_vic = keyword('협박 대화', attack_conv)

협박 대화의 가해자 : dict_keys(['내가', '지금', '니가', '진짜', '그럼', '그냥', '아니', '그래', '우리', '죽여', '너가', '당장', '어떻게', '사람', '빨리', '너도', '있어', '하나', '죽어', '나도', '그렇게', '당신', '죽고', '이렇게', '그러', '여기', '없어', '어디', '알아', '이게', '전에', '나한테', '오늘', '같이', '자꾸', '하면', '이제', '아주', '정신', '가족', '무슨', '이거', '말이', '있는', '생각', '칼로', '네가', '한번', '너네', '않으면', '죽이', '나랑', '새끼', '뭐가', '뭐?', '죽는', '하지', '다시', '헤어', '거야', '정말', '너는', '보고', '나는', '그런', '하는', '아직', '근데', '장기', '이런', '들어', '얼마나', '그게', '바로', '죽을', '너희', '말을', '이미', '너무', '다른', '조용히', '잘못', '하고', '해.', '계속', '집에', '니네', '싶어', '죽일', '알고', '소리', '수가', '좋은', '너를', '누가', '모르', '목숨', '엄마', '신고', '가만'])

협박 대화의 피해자 : dict_keys(['제발', '아니', '내가', '제가', '정말', '무슨', '지금', '진짜', '우리', '죄송', '살려', '어떻게', '그래', '그러', '그게', '잘못', '경찰에', '너무', '그렇게', '이제', '그건', '그냥', '그만', '그런', '네?', '이거', '사람', '미안', '이러', '알겠', '건들지', '한번만', '신고', '나도', '선생님', '조금만', '시간을', '저는', '나는', '없어', '협박', '당장', '그럼', '니가', '너가', '당신', '네.', '다시', '빨리', '주세요', '자꾸', '이렇게', '여기', '하지', '어디', '저도

In [41]:
other_att, other_vic = keyword('기타 괴롭힘 대화', attack_conv)

기타 괴롭힘 대화의 가해자 : dict_keys(['아니', '내가', '진짜', '그냥', '니가', '그래', '그럼', '우리', '여기', '그렇게', '지금', '너무', '무슨', '너가', '어떻게', '이거', '빨리', '사람', '그러', '어디', '얼굴', '이렇게', '뭐가', '너는', '냄새', '이게', '그런', '엄마', '새끼', '장애인', '친구', '돼지', '하는', '오늘', '근데', '나는', '그거', '하고', '없어', '애들', '기분', '너도', '뭐야', '말이', '못생', '같이', '너네', '저기', '다른', '이런', '다시', '알아', '존나', '제가', '누가', '그만', '있어', '다니', '뭐?', '남자', '집에', '너랑', '생각', '부모', '당장', '이제', '싫어', '따라', '하지', '환불', '나도', '학교', '없는', '하나', '찐따', '그게', '것도', '그리고', '하면', '말고', '에이', '거지', '맨날', '다리', '얼마나', '같은', '공부', '완전', '아가씨', '너한테', '사장', '좋아', '소리', '머리', '그건', '병신', '못하', '있는', '그니까', '앞으로'])

기타 괴롭힘 대화의 피해자 : dict_keys(['아니', '내가', '고객님', '무슨', '제발', '그만', '손님', '제가', '진짜', '지금', '너무', '죄송', '그런', '네?', '그렇게', '우리', '그게', '하지', '그래', '미안', '그냥', '말이', '나도', '나한테', '그러', '정말', '네.', '어떻게', '저희', '아.', '이거', '그건', '응?', '여기', '하는', '그럼', '자꾸', '사람', '너네', '있어', '오늘', '너가', '엄마', '니가', '혹시', '왜그', '선생님', '나는', '기분', '없어', '이제', '다른', '이렇게', '말을', '다

위 키워드를 바탕으로 중복을 제거한 가해자들의 키워드를 따로 저장해줌.

피해자는 대부분 `네? 죄송합니다. 미안 ` 등의 키워드가 나와서 비슷한 모습을 볼 수 있음.

공격적인 내용의 대부분은 가해자가 하는 말에의해 결정된다고 볼 수 있음.

In [42]:
att_keyword = extort_att + work_att + threat_att + other_att

In [44]:
# 중복 제거
att_keyword = list(set(att_keyword))
len(att_keyword)

62

In [79]:
def select_normal_conversation(sen, att_keyword, threshold=0.6):

  client = []
  counselor = []

  sen_split = sen.split('\n')
  for sen_ in sen_split:
    if sen_[0] == '1':
      client.append(sen_[2:])
    else:
      client.append(sen_[2:])
  
  from krwordrank.sentence import summarize_with_sentences

  texts = client
  keywords_client, sents = summarize_with_sentences(texts, num_keywords=10, 
                                           num_keysents=5, max_length=10, diversity=0.3)

  keywords_client = list(keywords_client.keys())
  keywords_att = att_keyword

  length = len(keywords_client)

  clean_keywords = []
  for word in keywords_client:
    if not word in keywords_att:
      clean_keywords.append(word)
  
  clean_ratio = len(clean_keywords) / length
  if clean_ratio > threshold:
    return True
  else:
    return False

In [80]:
normal_idx = []

for i in range(len(normal_conv)):
  sen = normal_conv.iloc[i][2]
  try:
    if select_normal_conversation(sen, att_keyword):
      normal_idx.append(i)
  except:
    continue

In [69]:
len(normal_idx)

13830

In [73]:
filtered_normal_conv = normal_conv.iloc[normal_idx]
filtered_normal_conv

,type,non_label_sentence,label_sentence,class,binary_class
0,고객센터,반갑습니다 입니다 무엇을 도와드릴까요\n여보세요 네 안녕하세요 거기 맞나요\n연결 ...,1:반갑습니다 입니다 무엇을 도와드릴까요\n2:여보세요 네 안녕하세요 거기 맞나요\...,일반 대화,일반 대화
1,고객센터,조직도 확인 후 연락드렸습니다\n정확한 문의는 부서에 문의를 해보시면 좋으실 듯 합...,1: 조직도 확인 후 연락드렸습니다\n1:정확한 문의는 부서에 문의를 해보시면 좋으...,일반 대화,일반 대화
2,고객센터,반갑습니다 입니다 무엇을 도와드릴까요\n제가 주택 임대차로 분쟁조정위원회쪽을 통화를...,1:반갑습니다 입니다 무엇을 도와드릴까요\n2:제가 주택 임대차로 분쟁조정위원회쪽을...,일반 대화,일반 대화
3,고객센터,반갑습니다 입니다 무엇을 도와 드릴까요\n여기요 인가요\n네 맞습니다 입니다\n네 ...,1:반갑습니다 입니다 무엇을 도와 드릴까요\n2:여기요 인가요\n1:네 맞습니다 입...,일반 대화,일반 대화
4,고객센터,네 입니다\n안녕하세요 입니다\n네 안녕하세요 무슨 일이시죠\n안녕하세요 주사님 한...,2:네 입니다\n1:안녕하세요 입니다\n2:네 안녕하세요 무슨 일이시죠\n1:안녕하...,일반 대화,일반 대화
...,...,...,...,...,...
47980,카카오,왜 아직도 덥지 머리 박박 밀어버릴까 봐\n너무 극단적인 거 아니가 갑자기 대머리 ...,1:왜 아직도 덥지 머리 박박 밀어버릴까 봐\n2:너무 극단적인 거 아니가 갑자기 ...,일반 대화,일반 대화
47981,카카오,따릉이 타고 출퇴근 하면 어떨까\n사지가 다 떨어지는 느낌이겠지\n슬프네 엄청 힘들...,1:따릉이 타고 출퇴근 하면 어떨까\n2:사지가 다 떨어지는 느낌이겠지\n1: 슬프...,일반 대화,일반 대화
47987,카카오,저녁에 일정 있어 피방 가자\n나 요즘에 하는 게임 없는데 뭐 하려고\n오버워치 나...,1:저녁에 일정 있어 피방 가자\n2:나 요즘에 하는 게임 없는데 뭐 하려고\n1:...,일반 대화,일반 대화
47989,카카오,오늘 친구네 강아지 산책하는데 따라갔다 왔다\n와 부러워 전에 그 바닷가 사는 애\...,1:오늘 친구네 강아지 산책하는데 따라갔다 왔다\n2:와 부러워 전에 그 바닷가 사...,일반 대화,일반 대화


In [88]:
cs_data = filtered_normal_conv[filtered_normal_conv['type']=='고객센터']
cs_data

,type,non_label_sentence,label_sentence,class,binary_class
0,고객센터,반갑습니다 입니다 무엇을 도와드릴까요\n여보세요 네 안녕하세요 거기 맞나요\n연결 ...,1:반갑습니다 입니다 무엇을 도와드릴까요\n2:여보세요 네 안녕하세요 거기 맞나요\...,일반 대화,일반 대화
1,고객센터,조직도 확인 후 연락드렸습니다\n정확한 문의는 부서에 문의를 해보시면 좋으실 듯 합...,1: 조직도 확인 후 연락드렸습니다\n1:정확한 문의는 부서에 문의를 해보시면 좋으...,일반 대화,일반 대화
2,고객센터,반갑습니다 입니다 무엇을 도와드릴까요\n제가 주택 임대차로 분쟁조정위원회쪽을 통화를...,1:반갑습니다 입니다 무엇을 도와드릴까요\n2:제가 주택 임대차로 분쟁조정위원회쪽을...,일반 대화,일반 대화
3,고객센터,반갑습니다 입니다 무엇을 도와 드릴까요\n여기요 인가요\n네 맞습니다 입니다\n네 ...,1:반갑습니다 입니다 무엇을 도와 드릴까요\n2:여기요 인가요\n1:네 맞습니다 입...,일반 대화,일반 대화
4,고객센터,네 입니다\n안녕하세요 입니다\n네 안녕하세요 무슨 일이시죠\n안녕하세요 주사님 한...,2:네 입니다\n1:안녕하세요 입니다\n2:네 안녕하세요 무슨 일이시죠\n1:안녕하...,일반 대화,일반 대화
...,...,...,...,...,...
29940,고객센터,소중히 모시겠습니다 상담원 입니다\n네 안녕하세요 문의 사항이 있어서요\n네 무엇을...,1: 소중히 모시겠습니다 상담원 입니다\n2: 네 안녕하세요 문의 사항이 있어서요\...,일반 대화,일반 대화
29952,고객센터,상쾌한 아침입니다 입니다\n안녕하세요 모바일 웹에서 궁금한게 있어서요\n네 안녕하세...,1:상쾌한 아침입니다 입니다\n2:안녕하세요 모바일 웹에서 궁금한게 있어서요\n1:...,일반 대화,일반 대화
29967,고객센터,즐거운 하루의 시작 저희 과 해주셔서 감사합니다\n무엇을 도와드릴까요\n네 안녕하세...,1:즐거운 하루의 시작 저희 과 해주셔서 감사합니다\n1:무엇을 도와드릴까요\n2:...,일반 대화,일반 대화
29978,고객센터,반갑습니다 상담원 입니다\n안녕하세요 문의가 있어서 연락드렸습니다\n네 어떤 문의가...,1:반갑습니다 상담원 입니다\n2:안녕하세요 문의가 있어서 연락드렸습니다\n1:네 ...,일반 대화,일반 대화


In [89]:
kakao_data = filtered_normal_conv[filtered_normal_conv['type'] == '카카오']
kakao_data

,type,non_label_sentence,label_sentence,class,binary_class
29995,카카오,아니야 싫어\n난 자격증 공부나 할래\n자격증 공부는 좀 돼가\n요즘 계속 못하다가...,1:아니야 싫어\n1:난 자격증 공부나 할래\n2:자격증 공부는 좀 돼가\n1:요즘...,일반 대화,일반 대화
29997,카카오,니는 예능 프로그램 챙겨 보는 거 뭐 있어\n음 대탈출 챙겨 봤었어요 하하\n나는 ...,1:니는 예능 프로그램 챙겨 보는 거 뭐 있어\n2:음 대탈출 챙겨 봤었어요 하하\...,일반 대화,일반 대화
29998,카카오,오늘 에서 점심은 뭐 먹었어\n오늘은 그냥 지나가다가 햄버거 먹었어\n올 버거 어디...,1:오늘 에서 점심은 뭐 먹었어\n2:오늘은 그냥 지나가다가 햄버거 먹었어\n1:올...,일반 대화,일반 대화
30001,카카오,근데 요즘은 만화도 많이 봐\n진짜 무슨 만화\n나는 짱구 좋아함\n나도 짱구도 좋...,1:근데 요즘은 만화도 많이 봐\n2:진짜 무슨 만화\n2:나는 짱구 좋아함\n1:...,일반 대화,일반 대화
30002,카카오,저는 히트텍을 꺼내 입었습니다 지금 날씨에\n우리 에도 보니깐 나이 많으신 분들 내...,1:저는 히트텍을 꺼내 입었습니다 지금 날씨에\n2: 우리 에도 보니깐 나이 많으신...,일반 대화,일반 대화
...,...,...,...,...,...
47980,카카오,왜 아직도 덥지 머리 박박 밀어버릴까 봐\n너무 극단적인 거 아니가 갑자기 대머리 ...,1:왜 아직도 덥지 머리 박박 밀어버릴까 봐\n2:너무 극단적인 거 아니가 갑자기 ...,일반 대화,일반 대화
47981,카카오,따릉이 타고 출퇴근 하면 어떨까\n사지가 다 떨어지는 느낌이겠지\n슬프네 엄청 힘들...,1:따릉이 타고 출퇴근 하면 어떨까\n2:사지가 다 떨어지는 느낌이겠지\n1: 슬프...,일반 대화,일반 대화
47987,카카오,저녁에 일정 있어 피방 가자\n나 요즘에 하는 게임 없는데 뭐 하려고\n오버워치 나...,1:저녁에 일정 있어 피방 가자\n2:나 요즘에 하는 게임 없는데 뭐 하려고\n1:...,일반 대화,일반 대화
47989,카카오,오늘 친구네 강아지 산책하는데 따라갔다 왔다\n와 부러워 전에 그 바닷가 사는 애\...,1:오늘 친구네 강아지 산책하는데 따라갔다 왔다\n2:와 부러워 전에 그 바닷가 사...,일반 대화,일반 대화


In [75]:
filtered_normal_conv.to_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/real_norm_data', index=True)

### 3. 비율에 따라 일반대화 추가

먼저 카카오톡 대화와 고객센터 대화의 비율은 3:1로 정했습니다.

기존의 공격대화에서 볼 때 직장 내 괴롭힘 클래스에 해당하는 대화를 제외하고는 대부분 가벼운 말투(야, 너, 가져와, 해라 등등)입니다.

추가할 일반 데이터인 카카오톡 데이터와 고객센터 데이터를 비교해보면 카카오톡 데이터가 비교적 가벼운 말투로 이루어져있고, 고객센터 대화가 조금 더 딱딱한 말투로 이루어져 있기 때문에 이에 맞춰서 데이터의 비율을 정했습니다.

먼저 공격대화를 일반 대화와 비슷하게 한글 숫자 제외하고 전부 전처리 해줍니다.

In [81]:
attack_conv

,class,label_sentence,non_label_sentence,binary_class
0,직장 내 괴롭힘 대화,1:길동씨 이번에 이것좀 처리해요\n2:이거 제가 한게 아닌데요\n1:팀에서 내가 ...,길동 씨 이번에 이것 좀 처리해요\n이거 제가 한 게 아닌데요\n팀에서 내가 네가 ...,공격 대화
1,기타 괴롭힘 대화,1:야 너 앞니 왜 그렇게 튀어나왔냐?\n2:태어날때부터 그랬어 물어보지마.\n1:...,야 너 앞니 왜 그렇게 튀어나왔냐?\n태어날 때부터 그랬어 물어보지 마.\n아 그럼...,공격 대화
2,갈취 대화,1:원후야 학원 교재 사야되는데 8만원만 주면 안돼?\n2:미안 나 오늘은 진짜 돈...,원후야 학원 교재 사야 되는데 8만 원만 주면 안 돼?\n미안 나 오늘은 진짜 돈이...,공격 대화
3,갈취 대화,1:너 저번에 술먹은 날 기억해?\n2:아니 왜?\n1:야 300만원만 가져와.\n...,너 저번에 술 먹은 날 기억해?\n아니 왜?\n야 300만 원만 가져와.\n뭔 소리...,공격 대화
4,협박 대화,1:너 또 내말 무시하냐?\n1:이새끼 널 좆으로 보나본데?\n2:아냐 진짜 시간이...,너 또 내말 무시하냐?\n이 새끼 널 좆으로 보나 본데?\n아냐 진짜 시간이 없었어...,공격 대화
...,...,...,...,...
3908,기타 괴롭힘 대화,1:준하야 넌 대가리가 왜이렇게 크냐?\n2:내 머리가 뭐.\n1:밥먹으면 대가리만...,준하야 넌 대가리가 왜 이렇게 크냐?\n내 머리가 뭐.\n밥 먹으면 대가리만 크냐 ...,공격 대화
3909,갈취 대화,1:내가 지금 너 아들 김길준 데리고 있어. 살리고 싶으면 계좌에 1억만 보내\n2...,내가 지금 너 아들 김길준 데리고 있어. 살리고 싶으면 계좌에 1억만 보내\n예.?...,공격 대화
3910,직장 내 괴롭힘 대화,1:나는 씨 같은 사람 보면 참 신기하더라. 어떻게 저렇게 살지.\n1:왜 그래. ...,나는 씨 같은 사람 보면 참 신기하더라. 어떻게 저렇게 살지.\n왜 그래. 들리겠어...,공격 대화
3911,갈취 대화,1:누구맘대로 여기서 장사하래?\n2:이게 무슨일입니까?\n1:남의 구역에서 장사하...,누구 맘대로 여기서 장사하래?\n이게 무슨 일입니까?\n남의 구역에서 장사하려면 자...,공격 대화


In [82]:
def preprocess(sen):
  sen = sen.split('\n')
  
  clean_sen = []
  for i in sen:
    i = i.strip()
    i = re.sub(r'[^0-9가-힣:]', ' ', i)
    i = i.strip()
    clean_sen.append(i)

  return '\n'.join(clean_sen)

In [83]:
attack_conv['label_sentence'] = attack_conv['label_sentence'].apply(lambda x : preprocess(x))
attack_conv

,class,label_sentence,non_label_sentence,binary_class
0,직장 내 괴롭힘 대화,1:길동씨 이번에 이것좀 처리해요\n2:이거 제가 한게 아닌데요\n1:팀에서 내가 ...,길동 씨 이번에 이것 좀 처리해요\n이거 제가 한 게 아닌데요\n팀에서 내가 네가 ...,공격 대화
1,기타 괴롭힘 대화,1:야 너 앞니 왜 그렇게 튀어나왔냐\n2:태어날때부터 그랬어 물어보지마\n1:아 ...,야 너 앞니 왜 그렇게 튀어나왔냐?\n태어날 때부터 그랬어 물어보지 마.\n아 그럼...,공격 대화
2,갈취 대화,1:원후야 학원 교재 사야되는데 8만원만 주면 안돼\n2:미안 나 오늘은 진짜 돈이...,원후야 학원 교재 사야 되는데 8만 원만 주면 안 돼?\n미안 나 오늘은 진짜 돈이...,공격 대화
3,갈취 대화,1:너 저번에 술먹은 날 기억해\n2:아니 왜\n1:야 300만원만 가져와\n2:뭔...,너 저번에 술 먹은 날 기억해?\n아니 왜?\n야 300만 원만 가져와.\n뭔 소리...,공격 대화
4,협박 대화,1:너 또 내말 무시하냐\n1:이새끼 널 좆으로 보나본데\n2:아냐 진짜 시간이 없...,너 또 내말 무시하냐?\n이 새끼 널 좆으로 보나 본데?\n아냐 진짜 시간이 없었어...,공격 대화
...,...,...,...,...
3908,기타 괴롭힘 대화,1:준하야 넌 대가리가 왜이렇게 크냐\n2:내 머리가 뭐\n1:밥먹으면 대가리만 크...,준하야 넌 대가리가 왜 이렇게 크냐?\n내 머리가 뭐.\n밥 먹으면 대가리만 크냐 ...,공격 대화
3909,갈취 대화,1:내가 지금 너 아들 김길준 데리고 있어 살리고 싶으면 계좌에 1억만 보내\n2...,내가 지금 너 아들 김길준 데리고 있어. 살리고 싶으면 계좌에 1억만 보내\n예.?...,공격 대화
3910,직장 내 괴롭힘 대화,1:나는 씨 같은 사람 보면 참 신기하더라 어떻게 저렇게 살지\n1:왜 그래 들...,나는 씨 같은 사람 보면 참 신기하더라. 어떻게 저렇게 살지.\n왜 그래. 들리겠어...,공격 대화
3911,갈취 대화,1:누구맘대로 여기서 장사하래\n2:이게 무슨일입니까\n1:남의 구역에서 장사하려면...,누구 맘대로 여기서 장사하래?\n이게 무슨 일입니까?\n남의 구역에서 장사하려면 자...,공격 대화


In [84]:
attack_conv['non_label_sentence'] = attack_conv['non_label_sentence'].apply(lambda x : preprocess(x))
attack_conv

,class,label_sentence,non_label_sentence,binary_class
0,직장 내 괴롭힘 대화,1:길동씨 이번에 이것좀 처리해요\n2:이거 제가 한게 아닌데요\n1:팀에서 내가 ...,길동 씨 이번에 이것 좀 처리해요\n이거 제가 한 게 아닌데요\n팀에서 내가 네가 ...,공격 대화
1,기타 괴롭힘 대화,1:야 너 앞니 왜 그렇게 튀어나왔냐\n2:태어날때부터 그랬어 물어보지마\n1:아 ...,야 너 앞니 왜 그렇게 튀어나왔냐\n태어날 때부터 그랬어 물어보지 마\n아 그럼 태...,공격 대화
2,갈취 대화,1:원후야 학원 교재 사야되는데 8만원만 주면 안돼\n2:미안 나 오늘은 진짜 돈이...,원후야 학원 교재 사야 되는데 8만 원만 주면 안 돼\n미안 나 오늘은 진짜 돈이 ...,공격 대화
3,갈취 대화,1:너 저번에 술먹은 날 기억해\n2:아니 왜\n1:야 300만원만 가져와\n2:뭔...,너 저번에 술 먹은 날 기억해\n아니 왜\n야 300만 원만 가져와\n뭔 소리야 ...,공격 대화
4,협박 대화,1:너 또 내말 무시하냐\n1:이새끼 널 좆으로 보나본데\n2:아냐 진짜 시간이 없...,너 또 내말 무시하냐\n이 새끼 널 좆으로 보나 본데\n아냐 진짜 시간이 없었어 미...,공격 대화
...,...,...,...,...
3908,기타 괴롭힘 대화,1:준하야 넌 대가리가 왜이렇게 크냐\n2:내 머리가 뭐\n1:밥먹으면 대가리만 크...,준하야 넌 대가리가 왜 이렇게 크냐\n내 머리가 뭐\n밥 먹으면 대가리만 크냐 너는...,공격 대화
3909,갈취 대화,1:내가 지금 너 아들 김길준 데리고 있어 살리고 싶으면 계좌에 1억만 보내\n2...,내가 지금 너 아들 김길준 데리고 있어 살리고 싶으면 계좌에 1억만 보내\n예 ...,공격 대화
3910,직장 내 괴롭힘 대화,1:나는 씨 같은 사람 보면 참 신기하더라 어떻게 저렇게 살지\n1:왜 그래 들...,나는 씨 같은 사람 보면 참 신기하더라 어떻게 저렇게 살지\n왜 그래 들리겠어\...,공격 대화
3911,갈취 대화,1:누구맘대로 여기서 장사하래\n2:이게 무슨일입니까\n1:남의 구역에서 장사하려면...,누구 맘대로 여기서 장사하래\n이게 무슨 일입니까\n남의 구역에서 장사하려면 자릿세...,공격 대화


1. 일반 대화 1000개 추가 (카카오톡 750 : 고객센터 250)

In [92]:
kakao_750 = kakao_data.sample(750)
kakao_750

,type,non_label_sentence,label_sentence,class,binary_class
38835,카카오,아오 요즘 맨날 늦게 자고 일찍 일어나니까 피부가 꺼슬꺼슬 다크 서클 잔뜩 내려옴\...,1:아오 요즘 맨날 늦게 자고 일찍 일어나니까 피부가 꺼슬꺼슬 다크 서클 잔뜩 내려...,일반 대화,일반 대화
30312,카카오,고양이를 키우는 건 정말 즐거운 일이야 감자를 캐자\n우리집 고양이는 보일러 틀어주...,1:고양이를 키우는 건 정말 즐거운 일이야 감자를 캐자\n2:우리집 고양이는 보일러...,일반 대화,일반 대화
37295,카카오,조금 전에 뉴스 본 거 있어\n아니 뉴스에 무슨 기사 떴어\n완전 조금 전에 대박 ...,1:조금 전에 뉴스 본 거 있어\n2:아니 뉴스에 무슨 기사 떴어\n1:완전 조금 ...,일반 대화,일반 대화
43401,카카오,난 강아지 같은 푸근한 인상이 좋아\n오 나는 원래 고양이 상 좋아했는데 나도 강아...,1:난 강아지 같은 푸근한 인상이 좋아\n2:오 나는 원래 고양이 상 좋아했는데 나...,일반 대화,일반 대화
43726,카카오,니는 폰 게임 자주 하는 편이가\n가끔 삘 받으면 하긴 하는데 거의 안 하는 편 너...,1:니는 폰 게임 자주 하는 편이가\n2:가끔 삘 받으면 하긴 하는데 거의 안 하는...,일반 대화,일반 대화
...,...,...,...,...,...
31898,카카오,반려동물로 뭐 키워 보고 싶어\n나는 우리집에 강아지 있어 몽실이\n몽실이 만 들어...,1:반려동물로 뭐 키워 보고 싶어\n2:나는 우리집에 강아지 있어 몽실이\n1:몽실...,일반 대화,일반 대화
44290,카카오,난 그래서 지금 결혼 생각 없이 강아지랑 즐겁게 살고 있다\n아 좋다 좋아\n강아지...,1:난 그래서 지금 결혼 생각 없이 강아지랑 즐겁게 살고 있다\n2:아 좋다 좋아\...,일반 대화,일반 대화
38491,카카오,니는 형제 있나 외동이가\n나도 너처럼 오빠 있어 하하 3살 차이\n오 오빠 있구나...,1:니는 형제 있나 외동이가\n2:나도 너처럼 오빠 있어 하하 3살 차이\n1:오 ...,일반 대화,일반 대화
30705,카카오,형은 형제 있으면 어떨 것 같은데\n난 누나 밖에 없어서 형이나 남동생 있었으면 좋...,1:형은 형제 있으면 어떨 것 같은데\n2:난 누나 밖에 없어서 형이나 남동생 있었...,일반 대화,일반 대화


In [93]:
cs_250 = cs_data.sample(250)
cs_250

,type,non_label_sentence,label_sentence,class,binary_class
20981,고객센터,안녕하십니까 친절한 상담원 입니다\n수고하십니다 다름아니라 궁금한 게 있어서 전화드...,1:안녕하십니까 친절한 상담원 입니다\n2:수고하십니다 다름아니라 궁금한 게 있어서...,일반 대화,일반 대화
1666,고객센터,반갑습니다 상담원 입니다\n전입 신고는 어떻게 해야 하나요\n신 거주지에 전입할 날...,1:반갑습니다 상담원 입니다\n2:전입 신고는 어떻게 해야 하나요\n1:신 거주지에...,일반 대화,일반 대화
16456,고객센터,안녕하세요 의 입니다\n저 주문하면서 쿠폰을 썼는데요\n네 쿠폰 적용하셨구요\n네 ...,1:안녕하세요 의 입니다\n2:저 주문하면서 쿠폰을 썼는데요\n1:네 쿠폰 적용하셨...,일반 대화,일반 대화
4004,고객센터,안녕하세요 무엇을 도와드릴까요\n운행제한차량 허가절차 및 신청요령을 알고 싶어서요\...,1:안녕하세요 무엇을 도와드릴까요\n2:운행제한차량 허가절차 및 신청요령을 알고 싶...,일반 대화,일반 대화
24402,고객센터,안녕하세요 입니다 무엇을 도와드릴까요\n네 안녕하세요 모의고사 중에 보면 온라인 모...,1:안녕하세요 입니다 무엇을 도와드릴까요\n2:네 안녕하세요 모의고사 중에 보면 온...,일반 대화,일반 대화
...,...,...,...,...,...
9963,고객센터,반갑습니다 입니다\n예 텔레비전 문의거든요\n텔레비전이 어제까지 멀쩡했었는데 아침에...,1:반갑습니다 입니다\n2:예 텔레비전 문의거든요\n2:텔레비전이 어제까지 멀쩡했었...,일반 대화,일반 대화
9066,고객센터,최선을 다하겠습니다 입니다\n저희 아들이 휴가를 나와야 하는데 못 나오고 있어요\n...,1:최선을 다하겠습니다 입니다\n2:저희 아들이 휴가를 나와야 하는데 못 나오고 있...,일반 대화,일반 대화
16642,고객센터,안녕하세요 고객 센터입니다\n어떤 도움이 필요하세요\n카드 결제 후 취소했거든요\n...,1:안녕하세요 고객 센터입니다\n1:어떤 도움이 필요하세요\n2:카드 결제 후 취소...,일반 대화,일반 대화
8143,고객센터,네 안녕하세요 입니다\n네 안녕하세요 님\n음성 터미널에서 천안까지 대중교통 경로 ...,1:네 안녕하세요 입니다\n2:네 안녕하세요 님\n1:음성 터미널에서 천안까지 대중...,일반 대화,일반 대화


In [94]:
norm_1000 = pd.concat([kakao_750, cs_250], ignore_index=True)
norm_1000

,type,non_label_sentence,label_sentence,class,binary_class
0,카카오,아오 요즘 맨날 늦게 자고 일찍 일어나니까 피부가 꺼슬꺼슬 다크 서클 잔뜩 내려옴\...,1:아오 요즘 맨날 늦게 자고 일찍 일어나니까 피부가 꺼슬꺼슬 다크 서클 잔뜩 내려...,일반 대화,일반 대화
1,카카오,고양이를 키우는 건 정말 즐거운 일이야 감자를 캐자\n우리집 고양이는 보일러 틀어주...,1:고양이를 키우는 건 정말 즐거운 일이야 감자를 캐자\n2:우리집 고양이는 보일러...,일반 대화,일반 대화
2,카카오,조금 전에 뉴스 본 거 있어\n아니 뉴스에 무슨 기사 떴어\n완전 조금 전에 대박 ...,1:조금 전에 뉴스 본 거 있어\n2:아니 뉴스에 무슨 기사 떴어\n1:완전 조금 ...,일반 대화,일반 대화
3,카카오,난 강아지 같은 푸근한 인상이 좋아\n오 나는 원래 고양이 상 좋아했는데 나도 강아...,1:난 강아지 같은 푸근한 인상이 좋아\n2:오 나는 원래 고양이 상 좋아했는데 나...,일반 대화,일반 대화
4,카카오,니는 폰 게임 자주 하는 편이가\n가끔 삘 받으면 하긴 하는데 거의 안 하는 편 너...,1:니는 폰 게임 자주 하는 편이가\n2:가끔 삘 받으면 하긴 하는데 거의 안 하는...,일반 대화,일반 대화
...,...,...,...,...,...
995,고객센터,반갑습니다 입니다\n예 텔레비전 문의거든요\n텔레비전이 어제까지 멀쩡했었는데 아침에...,1:반갑습니다 입니다\n2:예 텔레비전 문의거든요\n2:텔레비전이 어제까지 멀쩡했었...,일반 대화,일반 대화
996,고객센터,최선을 다하겠습니다 입니다\n저희 아들이 휴가를 나와야 하는데 못 나오고 있어요\n...,1:최선을 다하겠습니다 입니다\n2:저희 아들이 휴가를 나와야 하는데 못 나오고 있...,일반 대화,일반 대화
997,고객센터,안녕하세요 고객 센터입니다\n어떤 도움이 필요하세요\n카드 결제 후 취소했거든요\n...,1:안녕하세요 고객 센터입니다\n1:어떤 도움이 필요하세요\n2:카드 결제 후 취소...,일반 대화,일반 대화
998,고객센터,네 안녕하세요 입니다\n네 안녕하세요 님\n음성 터미널에서 천안까지 대중교통 경로 ...,1:네 안녕하세요 입니다\n2:네 안녕하세요 님\n1:음성 터미널에서 천안까지 대중...,일반 대화,일반 대화


In [95]:
train_1000 = pd.concat([norm_1000.iloc[:,1:], attack_conv])
train_1000

,non_label_sentence,label_sentence,class,binary_class
0,아오 요즘 맨날 늦게 자고 일찍 일어나니까 피부가 꺼슬꺼슬 다크 서클 잔뜩 내려옴\...,1:아오 요즘 맨날 늦게 자고 일찍 일어나니까 피부가 꺼슬꺼슬 다크 서클 잔뜩 내려...,일반 대화,일반 대화
1,고양이를 키우는 건 정말 즐거운 일이야 감자를 캐자\n우리집 고양이는 보일러 틀어주...,1:고양이를 키우는 건 정말 즐거운 일이야 감자를 캐자\n2:우리집 고양이는 보일러...,일반 대화,일반 대화
2,조금 전에 뉴스 본 거 있어\n아니 뉴스에 무슨 기사 떴어\n완전 조금 전에 대박 ...,1:조금 전에 뉴스 본 거 있어\n2:아니 뉴스에 무슨 기사 떴어\n1:완전 조금 ...,일반 대화,일반 대화
3,난 강아지 같은 푸근한 인상이 좋아\n오 나는 원래 고양이 상 좋아했는데 나도 강아...,1:난 강아지 같은 푸근한 인상이 좋아\n2:오 나는 원래 고양이 상 좋아했는데 나...,일반 대화,일반 대화
4,니는 폰 게임 자주 하는 편이가\n가끔 삘 받으면 하긴 하는데 거의 안 하는 편 너...,1:니는 폰 게임 자주 하는 편이가\n2:가끔 삘 받으면 하긴 하는데 거의 안 하는...,일반 대화,일반 대화
...,...,...,...,...
3908,준하야 넌 대가리가 왜 이렇게 크냐\n내 머리가 뭐\n밥 먹으면 대가리만 크냐 너는...,1:준하야 넌 대가리가 왜이렇게 크냐\n2:내 머리가 뭐\n1:밥먹으면 대가리만 크...,기타 괴롭힘 대화,공격 대화
3909,내가 지금 너 아들 김길준 데리고 있어 살리고 싶으면 계좌에 1억만 보내\n예 ...,1:내가 지금 너 아들 김길준 데리고 있어 살리고 싶으면 계좌에 1억만 보내\n2...,갈취 대화,공격 대화
3910,나는 씨 같은 사람 보면 참 신기하더라 어떻게 저렇게 살지\n왜 그래 들리겠어\...,1:나는 씨 같은 사람 보면 참 신기하더라 어떻게 저렇게 살지\n1:왜 그래 들...,직장 내 괴롭힘 대화,공격 대화
3911,누구 맘대로 여기서 장사하래\n이게 무슨 일입니까\n남의 구역에서 장사하려면 자릿세...,1:누구맘대로 여기서 장사하래\n2:이게 무슨일입니까\n1:남의 구역에서 장사하려면...,갈취 대화,공격 대화


In [96]:
train_1000.to_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/train_1000.csv')

2. 일반 대화 4000개 추가 (카카오 3000 : 고객센터 1000)

In [98]:
kakao_3000 = kakao_data.sample(3000)
kakao_3000

,type,non_label_sentence,label_sentence,class,binary_class
32700,카카오,주택에 살아 아파트에 살아\n나는 아파트에 살고 있어\n아파트 어때\n난 프로 주택...,1:주택에 살아 아파트에 살아\n2:나는 아파트에 살고 있어\n1:아파트 어때\n1...,일반 대화,일반 대화
41438,카카오,와 오늘은 오랜만에 밥 좀 먹어야겠다\n요새 밥 안 먹나 뭐 먹고 살아\n요새는 집...,1:와 오늘은 오랜만에 밥 좀 먹어야겠다\n2:요새 밥 안 먹나 뭐 먹고 살아\n1...,일반 대화,일반 대화
47464,카카오,너 왜 나 두고 혼자 보이스 봤어 8ㅅ8\n안니이 요새 딱히 할 일 없어서 그냥 심...,1:너 왜 나 두고 혼자 보이스 봤어 8ㅅ8\n2:안니이 요새 딱히 할 일 없어서 ...,일반 대화,일반 대화
36060,카카오,마약은 몰라도 마약처럼 빠져드는 영화는 있지\n영화 코카인이라고 그거 한번 봐\n나...,1:마약은 몰라도 마약처럼 빠져드는 영화는 있지\n2:영화 코카인이라고 그거 한번 ...,일반 대화,일반 대화
36912,카카오,여행 이용해 본 적 있어\n엉 이용해 봤지 해외 놀러 갔을 ㅋ 대\n오 여행사 통해...,1:여행 이용해 본 적 있어\n2:엉 이용해 봤지 해외 놀러 갔을 ㅋ 대\n1:오 ...,일반 대화,일반 대화
...,...,...,...,...,...
41568,카카오,나 지금 터미널 가서 버스 타려고\n오늘 차 막히지 않을까\n그런가 아까도 막히긴 ...,1:나 지금 터미널 가서 버스 타려고\n2:오늘 차 막히지 않을까\n1:그런가 아까...,일반 대화,일반 대화
41287,카카오,우리 벅수가 고추를 먹었엉\n엥 벅수 침 질질 안 흘리더나\n어제 먹어서 오늘 보다...,1:우리 벅수가 고추를 먹었엉\n2:엥 벅수 침 질질 안 흘리더나\n1: 어제 먹어...,일반 대화,일반 대화
44543,카카오,너는 살면서 등산해본 적 있어\n나는 내가 개인적으로 등산해본 적은 없는 것 같아\...,1:너는 살면서 등산해본 적 있어\n2:나는 내가 개인적으로 등산해본 적은 없는 것...,일반 대화,일반 대화
39826,카카오,나 보이스 보고 왔음\n영화 재밌더라\n얼른 보고 오라구 다들\n야 언제적 보이스냐...,1:나 보이스 보고 왔음\n1:영화 재밌더라\n1:얼른 보고 오라구 다들\n2:야 ...,일반 대화,일반 대화


In [99]:
cs_1000 = cs_data.sample(1000)
cs_1000

,type,non_label_sentence,label_sentence,class,binary_class
25235,고객센터,정성을 다해 상담해 드리겠습니다 입니다\n강의 결제전에 먼저 들어볼 수 있나요\n네...,1:정성을 다해 상담해 드리겠습니다 입니다\n2:강의 결제전에 먼저 들어볼 수 있나...,일반 대화,일반 대화
12148,고객센터,안녕하세요 입니다\n우리 구입일 좀 확인해주세요 일 년 넘었나요\n네 구입일이 이천...,1:안녕하세요 입니다\n2:우리 구입일 좀 확인해주세요 일 년 넘었나요\n1:네 구...,일반 대화,일반 대화
25630,고객센터,행복 가득한 고객센터 상담원 입니다\n아 네 저는 이라고 합니다\n수강을 고민하고 ...,1:행복 가득한 고객센터 상담원 입니다\n2:아 네 저는 이라고 합니다\n2: 수강...,일반 대화,일반 대화
8253,고객센터,안녕하십니까 친절한 상담원 입니다\n예 수고가 많으십니다 제가 질문사항이 있어서 전...,1:안녕하십니까 친절한 상담원 입니다\n2:예 수고가 많으십니다 제가 질문사항이 있...,일반 대화,일반 대화
23189,고객센터,안녕하세요 입니다 전화 맞으십니까\n네 제가 인데요 확인해보셨나요\n네 현재 블랙만...,1:안녕하세요 입니다 전화 맞으십니까\n2:네 제가 인데요 확인해보셨나요\n1:네 ...,일반 대화,일반 대화
...,...,...,...,...,...
15391,고객센터,안녕하세요 상담원 입니다\n제가 쿠폰이 있는데요\n네 어떤 문의이실까요\n결제할 때...,1:안녕하세요 상담원 입니다\n2:제가 쿠폰이 있는데요\n1:네 어떤 문의이실까요\...,일반 대화,일반 대화
25312,고객센터,문의주셔서 감사합니다 고객센터 입니다\n안녕하세요 제가 공무원 7급 준비하고 있는데...,1: 문의주셔서 감사합니다 고객센터 입니다\n2: 안녕하세요 제가 공무원 7급 준비...,일반 대화,일반 대화
2319,고객센터,기분 좋은 아침입니다 상담원 입니다\n안녕하세요 여쭐 게 있어 전화드렸습니다\n네 ...,1:기분 좋은 아침입니다 상담원 입니다\n2:안녕하세요 여쭐 게 있어 전화드렸습니다...,일반 대화,일반 대화
28577,고객센터,고객센터 입니다 무엇을 도와드릴까요\n안녕하세요 할인되는 카드 좀 알려 주세요\n해...,1: 고객센터 입니다 무엇을 도와드릴까요\n2:안녕하세요 할인되는 카드 좀 알려 주...,일반 대화,일반 대화


In [100]:
norm_4000 = pd.concat([kakao_3000, cs_1000], ignore_index=True)
norm_4000

,type,non_label_sentence,label_sentence,class,binary_class
0,카카오,주택에 살아 아파트에 살아\n나는 아파트에 살고 있어\n아파트 어때\n난 프로 주택...,1:주택에 살아 아파트에 살아\n2:나는 아파트에 살고 있어\n1:아파트 어때\n1...,일반 대화,일반 대화
1,카카오,와 오늘은 오랜만에 밥 좀 먹어야겠다\n요새 밥 안 먹나 뭐 먹고 살아\n요새는 집...,1:와 오늘은 오랜만에 밥 좀 먹어야겠다\n2:요새 밥 안 먹나 뭐 먹고 살아\n1...,일반 대화,일반 대화
2,카카오,너 왜 나 두고 혼자 보이스 봤어 8ㅅ8\n안니이 요새 딱히 할 일 없어서 그냥 심...,1:너 왜 나 두고 혼자 보이스 봤어 8ㅅ8\n2:안니이 요새 딱히 할 일 없어서 ...,일반 대화,일반 대화
3,카카오,마약은 몰라도 마약처럼 빠져드는 영화는 있지\n영화 코카인이라고 그거 한번 봐\n나...,1:마약은 몰라도 마약처럼 빠져드는 영화는 있지\n2:영화 코카인이라고 그거 한번 ...,일반 대화,일반 대화
4,카카오,여행 이용해 본 적 있어\n엉 이용해 봤지 해외 놀러 갔을 ㅋ 대\n오 여행사 통해...,1:여행 이용해 본 적 있어\n2:엉 이용해 봤지 해외 놀러 갔을 ㅋ 대\n1:오 ...,일반 대화,일반 대화
...,...,...,...,...,...
3995,고객센터,안녕하세요 상담원 입니다\n제가 쿠폰이 있는데요\n네 어떤 문의이실까요\n결제할 때...,1:안녕하세요 상담원 입니다\n2:제가 쿠폰이 있는데요\n1:네 어떤 문의이실까요\...,일반 대화,일반 대화
3996,고객센터,문의주셔서 감사합니다 고객센터 입니다\n안녕하세요 제가 공무원 7급 준비하고 있는데...,1: 문의주셔서 감사합니다 고객센터 입니다\n2: 안녕하세요 제가 공무원 7급 준비...,일반 대화,일반 대화
3997,고객센터,기분 좋은 아침입니다 상담원 입니다\n안녕하세요 여쭐 게 있어 전화드렸습니다\n네 ...,1:기분 좋은 아침입니다 상담원 입니다\n2:안녕하세요 여쭐 게 있어 전화드렸습니다...,일반 대화,일반 대화
3998,고객센터,고객센터 입니다 무엇을 도와드릴까요\n안녕하세요 할인되는 카드 좀 알려 주세요\n해...,1: 고객센터 입니다 무엇을 도와드릴까요\n2:안녕하세요 할인되는 카드 좀 알려 주...,일반 대화,일반 대화


In [101]:
train_4000 = pd.concat([norm_4000.iloc[:,1:], attack_conv])
train_4000

,non_label_sentence,label_sentence,class,binary_class
0,주택에 살아 아파트에 살아\n나는 아파트에 살고 있어\n아파트 어때\n난 프로 주택...,1:주택에 살아 아파트에 살아\n2:나는 아파트에 살고 있어\n1:아파트 어때\n1...,일반 대화,일반 대화
1,와 오늘은 오랜만에 밥 좀 먹어야겠다\n요새 밥 안 먹나 뭐 먹고 살아\n요새는 집...,1:와 오늘은 오랜만에 밥 좀 먹어야겠다\n2:요새 밥 안 먹나 뭐 먹고 살아\n1...,일반 대화,일반 대화
2,너 왜 나 두고 혼자 보이스 봤어 8ㅅ8\n안니이 요새 딱히 할 일 없어서 그냥 심...,1:너 왜 나 두고 혼자 보이스 봤어 8ㅅ8\n2:안니이 요새 딱히 할 일 없어서 ...,일반 대화,일반 대화
3,마약은 몰라도 마약처럼 빠져드는 영화는 있지\n영화 코카인이라고 그거 한번 봐\n나...,1:마약은 몰라도 마약처럼 빠져드는 영화는 있지\n2:영화 코카인이라고 그거 한번 ...,일반 대화,일반 대화
4,여행 이용해 본 적 있어\n엉 이용해 봤지 해외 놀러 갔을 ㅋ 대\n오 여행사 통해...,1:여행 이용해 본 적 있어\n2:엉 이용해 봤지 해외 놀러 갔을 ㅋ 대\n1:오 ...,일반 대화,일반 대화
...,...,...,...,...
3908,준하야 넌 대가리가 왜 이렇게 크냐\n내 머리가 뭐\n밥 먹으면 대가리만 크냐 너는...,1:준하야 넌 대가리가 왜이렇게 크냐\n2:내 머리가 뭐\n1:밥먹으면 대가리만 크...,기타 괴롭힘 대화,공격 대화
3909,내가 지금 너 아들 김길준 데리고 있어 살리고 싶으면 계좌에 1억만 보내\n예 ...,1:내가 지금 너 아들 김길준 데리고 있어 살리고 싶으면 계좌에 1억만 보내\n2...,갈취 대화,공격 대화
3910,나는 씨 같은 사람 보면 참 신기하더라 어떻게 저렇게 살지\n왜 그래 들리겠어\...,1:나는 씨 같은 사람 보면 참 신기하더라 어떻게 저렇게 살지\n1:왜 그래 들...,직장 내 괴롭힘 대화,공격 대화
3911,누구 맘대로 여기서 장사하래\n이게 무슨 일입니까\n남의 구역에서 장사하려면 자릿세...,1:누구맘대로 여기서 장사하래\n2:이게 무슨일입니까\n1:남의 구역에서 장사하려면...,갈취 대화,공격 대화


In [102]:
train_4000.to_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/train_4000.csv')

3. 일반 대화 10000개 추가 (카카오 7500 : 고객센터 2500)

In [103]:
kakao_7500 = kakao_data.sample(7500)
kakao_7500

,type,non_label_sentence,label_sentence,class,binary_class
46304,카카오,너 제주도 여행 자주 가봤어\n제주도 몇 번 가보긴 했지 우리 수학여행도 제주도 갔...,1:너 제주도 여행 자주 가봤어\n2:제주도 몇 번 가보긴 했지 우리 수학여행도 제...,일반 대화,일반 대화
30642,카카오,피부미용사 자격증 따 놓을 걸 그랬나\n피부미용사 자격증 그쪽에 관심이 있는 거야\...,1:피부미용사 자격증 따 놓을 걸 그랬나\n2:피부미용사 자격증 그쪽에 관심이 있는...,일반 대화,일반 대화
33099,카카오,대만 주상복합에 화재가 발생해서 46명이나 사망했데\n와 아까 봤는데 사망자가 더 ...,1:대만 주상복합에 화재가 발생해서 46명이나 사망했데\n2:와 아까 봤는데 사망자...,일반 대화,일반 대화
39187,카카오,언니는 차 언제 샀어요\n나는 지금 차는 5년 전에 샀어\n요즘 진짜 차가 너무 갖...,1:언니는 차 언제 샀어요\n2:나는 지금 차는 5년 전에 샀어\n1:요즘 진짜 차...,일반 대화,일반 대화
44508,카카오,저 요새 프리랜서로 일하잖아요\n네네 저도 프리랜서로 일해요\n근데 그래서 그런지 ...,1:저 요새 프리랜서로 일하잖아요\n2:네네 저도 프리랜서로 일해요\n1:근데 그래...,일반 대화,일반 대화
...,...,...,...,...,...
37900,카카오,아 아 너 진짜 소름 돋아 왜 그러는 거야 그러니까 차이고 다니지\n그러니까 여자친...,1:아 아 너 진짜 소름 돋아 왜 그러는 거야 그러니까 차이고 다니지\n2:그러니까...,일반 대화,일반 대화
41073,카카오,비행기 타고 출퇴근해 보는 게 소원이얌\n얼마나 재밌을까\n아악 너무 재밌겠다\n진...,1:비행기 타고 출퇴근해 보는 게 소원이얌\n1:얼마나 재밌을까\n2:아악 너무 재...,일반 대화,일반 대화
42959,카카오,아 하긴 너 군대 다녀오면 또 이직도 해야지\n응응 난 일본기업으로 이직 하려고 공...,1:아 하긴 너 군대 다녀오면 또 이직도 해야지\n2:응응 난 일본기업으로 이직 하...,일반 대화,일반 대화
37684,카카오,울 동네 길냥이들 중에 개냥이 엄청 많아여 하하\n아 전에 아침 조깅 갔다가 고양이...,1:울 동네 길냥이들 중에 개냥이 엄청 많아여 하하\n2:아 전에 아침 조깅 갔다가...,일반 대화,일반 대화


In [104]:
cs_2500 = cs_data.sample(2500)
cs_2500

,type,non_label_sentence,label_sentence,class,binary_class
24002,고객센터,안녕하세요 입니다 무엇을 도와드릴까요\n네 수고하십니다 모의고사 해설강의는 어떻게 ...,1:안녕하세요 입니다 무엇을 도와드릴까요\n2:네 수고하십니다 모의고사 해설강의는 ...,일반 대화,일반 대화
16056,고객센터,지금 저희가 전달한 상태인데 그쪽에서 연락이 안 가네요\n죄송합니다 잘 안들립니다\...,1:지금 저희가 전달한 상태인데 그쪽에서 연락이 안 가네요\n2:죄송합니다 잘 안들...,일반 대화,일반 대화
1073,고객센터,전화 주셔서 감사합니다 입니다\n궁금한 게 있어서 전화드렸어요\n네 선생님 어떤 내...,1:전화 주셔서 감사합니다 입니다\n2:궁금한 게 있어서 전화드렸어요\n1:네 선생...,일반 대화,일반 대화
12108,고객센터,반갑습니다 입니다\n구입해서 한 삼 사 개월 인가 삼 개월 후에 냉장고가 또 고장이...,1:반갑습니다 입니다\n2:구입해서 한 삼 사 개월 인가 삼 개월 후에 냉장고가 또...,일반 대화,일반 대화
28265,고객센터,감사합니다 입니다\n수고하십니다 아카데미 중에 인가 그걸 신청했는데요\n개인사정이 ...,1: 감사합니다 입니다\n2: 수고하십니다 아카데미 중에 인가 그걸 신청했는데요\n...,일반 대화,일반 대화
...,...,...,...,...,...
24584,고객센터,행복한 배움이 있는 입니다\n네 저는 중국 사람 이라고 하는데요\n네 말씀하세요\n...,1:행복한 배움이 있는 입니다\n2:네 저는 중국 사람 이라고 하는데요\n1:네 말...,일반 대화,일반 대화
662,고객센터,반갑습니다 입니다 무엇을 도와드릴까요\n네 여기가 홍성인데요 홍성경찰서 근처에 서점...,1:반갑습니다 입니다 무엇을 도와드릴까요\n2:네 여기가 홍성인데요 홍성경찰서 근처...,일반 대화,일반 대화
27009,고객센터,행복한 아침 입니다\n네 안녕하세요 문의드립니다\n네 무엇을 도와드릴까요\n이번 모...,1:행복한 아침 입니다\n2:네 안녕하세요 문의드립니다\n1:네 무엇을 도와드릴까요...,일반 대화,일반 대화
3921,고객센터,안녕하세요 상담원 입니다\n네 안녕하세요 문의할 게 있어서요\n네 어떤 문의이실까요...,1:안녕하세요 상담원 입니다\n2:네 안녕하세요 문의할 게 있어서요\n1:네 어떤 ...,일반 대화,일반 대화


In [105]:
norm_10000 = pd.concat([kakao_7500, cs_2500], ignore_index=True)
norm_10000

,type,non_label_sentence,label_sentence,class,binary_class
0,카카오,너 제주도 여행 자주 가봤어\n제주도 몇 번 가보긴 했지 우리 수학여행도 제주도 갔...,1:너 제주도 여행 자주 가봤어\n2:제주도 몇 번 가보긴 했지 우리 수학여행도 제...,일반 대화,일반 대화
1,카카오,피부미용사 자격증 따 놓을 걸 그랬나\n피부미용사 자격증 그쪽에 관심이 있는 거야\...,1:피부미용사 자격증 따 놓을 걸 그랬나\n2:피부미용사 자격증 그쪽에 관심이 있는...,일반 대화,일반 대화
2,카카오,대만 주상복합에 화재가 발생해서 46명이나 사망했데\n와 아까 봤는데 사망자가 더 ...,1:대만 주상복합에 화재가 발생해서 46명이나 사망했데\n2:와 아까 봤는데 사망자...,일반 대화,일반 대화
3,카카오,언니는 차 언제 샀어요\n나는 지금 차는 5년 전에 샀어\n요즘 진짜 차가 너무 갖...,1:언니는 차 언제 샀어요\n2:나는 지금 차는 5년 전에 샀어\n1:요즘 진짜 차...,일반 대화,일반 대화
4,카카오,저 요새 프리랜서로 일하잖아요\n네네 저도 프리랜서로 일해요\n근데 그래서 그런지 ...,1:저 요새 프리랜서로 일하잖아요\n2:네네 저도 프리랜서로 일해요\n1:근데 그래...,일반 대화,일반 대화
...,...,...,...,...,...
9995,고객센터,행복한 배움이 있는 입니다\n네 저는 중국 사람 이라고 하는데요\n네 말씀하세요\n...,1:행복한 배움이 있는 입니다\n2:네 저는 중국 사람 이라고 하는데요\n1:네 말...,일반 대화,일반 대화
9996,고객센터,반갑습니다 입니다 무엇을 도와드릴까요\n네 여기가 홍성인데요 홍성경찰서 근처에 서점...,1:반갑습니다 입니다 무엇을 도와드릴까요\n2:네 여기가 홍성인데요 홍성경찰서 근처...,일반 대화,일반 대화
9997,고객센터,행복한 아침 입니다\n네 안녕하세요 문의드립니다\n네 무엇을 도와드릴까요\n이번 모...,1:행복한 아침 입니다\n2:네 안녕하세요 문의드립니다\n1:네 무엇을 도와드릴까요...,일반 대화,일반 대화
9998,고객센터,안녕하세요 상담원 입니다\n네 안녕하세요 문의할 게 있어서요\n네 어떤 문의이실까요...,1:안녕하세요 상담원 입니다\n2:네 안녕하세요 문의할 게 있어서요\n1:네 어떤 ...,일반 대화,일반 대화


In [107]:
train_10000 = pd.concat([norm_10000.iloc[:, 1:], attack_conv], ignore_index=True)
train_10000

,non_label_sentence,label_sentence,class,binary_class
0,너 제주도 여행 자주 가봤어\n제주도 몇 번 가보긴 했지 우리 수학여행도 제주도 갔...,1:너 제주도 여행 자주 가봤어\n2:제주도 몇 번 가보긴 했지 우리 수학여행도 제...,일반 대화,일반 대화
1,피부미용사 자격증 따 놓을 걸 그랬나\n피부미용사 자격증 그쪽에 관심이 있는 거야\...,1:피부미용사 자격증 따 놓을 걸 그랬나\n2:피부미용사 자격증 그쪽에 관심이 있는...,일반 대화,일반 대화
2,대만 주상복합에 화재가 발생해서 46명이나 사망했데\n와 아까 봤는데 사망자가 더 ...,1:대만 주상복합에 화재가 발생해서 46명이나 사망했데\n2:와 아까 봤는데 사망자...,일반 대화,일반 대화
3,언니는 차 언제 샀어요\n나는 지금 차는 5년 전에 샀어\n요즘 진짜 차가 너무 갖...,1:언니는 차 언제 샀어요\n2:나는 지금 차는 5년 전에 샀어\n1:요즘 진짜 차...,일반 대화,일반 대화
4,저 요새 프리랜서로 일하잖아요\n네네 저도 프리랜서로 일해요\n근데 그래서 그런지 ...,1:저 요새 프리랜서로 일하잖아요\n2:네네 저도 프리랜서로 일해요\n1:근데 그래...,일반 대화,일반 대화
...,...,...,...,...
13908,준하야 넌 대가리가 왜 이렇게 크냐\n내 머리가 뭐\n밥 먹으면 대가리만 크냐 너는...,1:준하야 넌 대가리가 왜이렇게 크냐\n2:내 머리가 뭐\n1:밥먹으면 대가리만 크...,기타 괴롭힘 대화,공격 대화
13909,내가 지금 너 아들 김길준 데리고 있어 살리고 싶으면 계좌에 1억만 보내\n예 ...,1:내가 지금 너 아들 김길준 데리고 있어 살리고 싶으면 계좌에 1억만 보내\n2...,갈취 대화,공격 대화
13910,나는 씨 같은 사람 보면 참 신기하더라 어떻게 저렇게 살지\n왜 그래 들리겠어\...,1:나는 씨 같은 사람 보면 참 신기하더라 어떻게 저렇게 살지\n1:왜 그래 들...,직장 내 괴롭힘 대화,공격 대화
13911,누구 맘대로 여기서 장사하래\n이게 무슨 일입니까\n남의 구역에서 장사하려면 자릿세...,1:누구맘대로 여기서 장사하래\n2:이게 무슨일입니까\n1:남의 구역에서 장사하려면...,갈취 대화,공격 대화


In [108]:
train_10000.to_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/train_10000.csv')